#### Federated Learning 
Trains central models on decentralized data, hyper-personalized for users with minimum Latencies, while preserving privacy.

#### Importing Required Libraries

In [1]:
import os
import numpy as np
import torch, torchvision
import torch as T
import torch.nn as nn
import random
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from numpy import vstack
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.optim import SGD
from torch.nn import BCELoss

In [2]:
class PeopleDataset(T.utils.data.Dataset):
    def __init__(self, src_file, num_rows=None):
        df = pd.read_csv(src_file)
        df.drop(df.columns[[0]], axis=1, inplace=True)
        print(df.columns)
        df.Class = df.Class.astype('float64')
        y_tmp = df['Class'].values
        x_tmp = df.drop('Class', axis=1).values
                

        self.x_data = T.tensor(x_tmp,dtype=T.float64).to(device)
        self.y_data = T.tensor(y_tmp,dtype=T.float64).to(device)

        print(type(self.x_data))
        print(len(self.x_data))

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        preds = self.x_data[idx].type(T.FloatTensor)
        pol = self.y_data[idx].type(T.LongTensor)
        sample = [preds, pol]
        return sample

In [3]:
class MLPUpdated(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(30, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [4]:
federated_acc, federated_pre, federated_recall, federated_f1 = list(), list(), list(), list()
mp = {}

### Federated Learning

In [5]:
# Can be changed to 12,16,24,32
num_clients = 32
# Change it to 3, 6, 10, 16
num_selected = 16
num_rounds = 50
batch_size = 1024
epochs = 20
device = "cpu"
device = torch.device(device)
federated_acc, federated_pre, federated_recall, federated_f1 = list(), list(), list(), list()

### Train the model on decentralized or user data.

In [6]:
# Dividing the training data into num_clients, with each client having equal number of data
traindata = PeopleDataset('creditsampledata.csv')
print(len(traindata))
traindata_split = torch.utils.data.random_split(traindata, [int(len(traindata) / num_clients) for _ in range(num_clients)])
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]


test_file = 'creditsampledata.csv'
test_ds = PeopleDataset(test_file)
test_loader = T.utils.data.DataLoader(test_ds,batch_size=batch_size, shuffle=True)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
<class 'torch.Tensor'>
8000
8000
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
<class 'torch.Tensor'>
8000


### Model training on client or user data to evaluate and test the baseline performance.

In [8]:
def client_update(client_model, optimizer, train_loader, epoch=15):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = client_model(data)
            binary_loss = torch.nn.BCEWithLogitsLoss()
            target = target.unsqueeze(1)
            target = target.float()
            loss = binary_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [9]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [14]:
def test(global_model, test_loader):
    "The function test the global model on test data and returns test loss and test accuracy"
    model.eval()
    test_loss = 0
    correct = 0
    actuals, predictions = list(), list()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = global_model(data)
            binary_loss = torch.nn.BCEWithLogitsLoss()
            target = target.unsqueeze(1)
            target = target.float()
            test_loss += binary_loss(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            actual = target.numpy()
            pr = output.detach().numpy()
            pr = pr.round()
            predictions.append(pr)
            actuals.append(actual)

    test_loss /= len(test_loader.dataset)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    # calculate precision
    prescision = precision_score(actuals, predictions)
    # calculate recall
    recall = recall_score(actuals, predictions)
    # calculate f1
    f1 = f1_score(actuals, predictions)
    federated_acc.append(acc)
    federated_pre.append(prescision)
    federated_recall.append(recall)
    federated_f1.append(f1)
    print()
    print(confusion_matrix(actuals, predictions))
    return test_loss, acc, prescision, recall, f1

In [15]:
## Initializing models optimizer and global_model =  MLPUpdated
global_model = MLPUpdated().to(device)

## client models
client_models = [ MLPUpdated().to(device) for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ## initial synchronizing with global model 

## optimizers 
opt = [optim.SGD(model.parameters(), lr=0.01) for model in client_models]

In [16]:
## List containing info about learning
losses_train = []
losses_test = []
acc_train = []
acc_test = []
# Runnining FL

import time
start_time = time.time()
for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    losses_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    test_loss, acc, prescision, recall, f1= test(global_model, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f | test prescision: %0.3f | test recall: %0.3f | test f1: %0.3f' % (loss / num_selected, test_loss, acc, prescision, recall, f1))

print("--- %s seconds ---" % (time.time() - start_time))
# time[24]['fed'] = (time.time() - start_time)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.04it/s]



[[7973   10]
 [  17    0]]
0-th round
average train loss 0.695 | test loss 0.000694 | test acc: 0.997 | test prescision: 0.000 | test recall: 0.000 | test f1: 0.000


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.53it/s]



[[7977    6]
 [  17    0]]
1-th round
average train loss 0.693 | test loss 0.000694 | test acc: 0.997 | test prescision: 0.000 | test recall: 0.000 | test f1: 0.000


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.13it/s]


...


[[7983    0]
 [  17    0]]
47-th round
average train loss 0.693 | test loss 0.000693 | test acc: 0.998 | test prescision: 0.000 | test recall: 0.000 | test f1: 0.000


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.37it/s]



[[7982    1]
 [  17    0]]
48-th round
average train loss 0.693 | test loss 0.000693 | test acc: 0.998 | test prescision: 0.000 | test recall: 0.000 | test f1: 0.000


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  3.19it/s]



[[7982    1]
 [  17    0]]
49-th round
average train loss 0.693 | test loss 0.000693 | test acc: 0.998 | test prescision: 0.000 | test recall: 0.000 | test f1: 0.000
--- 281.23680448532104 seconds ---


In [17]:
print(len(federated_acc))
print(len(federated_f1))

50
50


In [18]:
mp[num_selected] = {'fed_acc': federated_acc, 'fed_pre': federated_pre, 'fed_recall': federated_recall, 'fed_f1': federated_f1}
print(mp)
print(len(mp))

{16: {'fed_acc': [0.996625, 0.997125, 0.997125, 0.997625, 0.997375, 0.997625, 0.9975, 0.997625, 0.997625, 0.997625, 0.99775, 0.9975, 0.997625, 0.997625, 0.997875, 0.997625, 0.997875, 0.99775, 0.997625, 0.997625, 0.99775, 0.99775, 0.99775, 0.997875, 0.997875, 0.997625, 0.997875, 0.997875, 0.997625, 0.997875, 0.99775, 0.99775, 0.997625, 0.997875, 0.997875, 0.99775, 0.997875, 0.997875, 0.997875, 0.997875, 0.99775, 0.997875, 0.99775, 0.99775, 0.9975, 0.9975, 0.99775, 0.997875, 0.99775, 0.99775], 'fed_pre': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'fed_recall': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 